# Take Home Project: Wrangling coal mine data

* This task is an example of the kind of work we do to make public energy data usable for analysis.
* We want to be able to explore your general approach together and see how you think about these kinds of problems.
* **Spend 2-4 hours working on it.** This doesn't have to happen all at once. We want you to have time to play with the data, step away from it to think, and then come back to it again.
* Feel free to use whatever documentation or online resources you would normally consult while working on a data wrangling problem.
* Feel free to use additional 3rd party libraries if you want to.  You should be able to install them from within the notebook using `!pip install packagename` or `!conda install packagename`

## Email us your notebook within a week.
* Send it to [hello@catalyst.coop](mailto:hello@catalyst.coop) (normally we'd have you make a PR but... we don't want everyone looking at each others solutions)
* We'll review your notebook and if it looks good, we'll reach out to schedule a longer conversation about it, and another technical interview.

## Some questions to keep in mind:
* What assumptions are you making about the data?
* Is the raw data well structured?
* How will you test whether / when those assumptions are valid?
* How would you / did you deal with the data that don’t conform to those assumptions?
* If there are records which can’t be reasonably cleaned automatically, but were high value in an advocacy context, how would you integrate manual cleaning into the automated process so that the manual effort is captured, and can be incrementally improved over time?
* What expectations do you have about the output data?
* What kind of data validation checks would you design to make sure that the output meets your expectations? These could be either integrated into the table transformation process, or run on the final output.
* How do you decide when data isn’t recoverable?
* How will you evaluate the completeness of the data that you’ve been able to extract?
* What kind of queries are you trying to make easy with the structure of the output data?
* What parts of this process might make sense to generalize / abstract for re-use in extracting, cleaning, and reorganizing data from other tables?

## Background on the MSHA Coal Mine Data

* The Mine Health and Safety Administration (MSHA) collects a variety of information about mines, incuding who owns them, what and how much they produce, mining methods used, environmental and safety violations, number of employees, ownership, and location.
* This information can be helpful for understanding the economic and environmental consequences of shutting down coal fired power plants. It's especially relevant right now, since the Inflation Reduction Act (IRA) provides tax benefits for clean energy projects in former coal communities. (You can read more about "energy and coal communitites" [here](https://www.resources.org/common-resources/what-is-an-energy-community/), but that's not required to answer this interview question.)
* The US Energy Information Administration's Form 923 (EIA 923) reports fuel deliveries to individual coal-fired power plants, including the fuel's cost, heat content, the quantity delivered, how it was transported, who they bought it from and what mine it came from.
* EIA includes some information about the mine in each fuel delivery record, and because multiple delieveries are typically reported from every mine, there's a lot of duplication and internal inconsistencies. In an ideal world, they would probably just refer to authoritative data about the mine that is collected by MSHA instead.
* We have already integrated some of this EIA 923 data into our Public Utility Data Liberation (PUDL) database.
* We've stored the messy, sometimes duplicative coal mine data in a table called `coalmine_eia923` ([data dictionary](https://catalystcoop-pudl.readthedocs.io/en/latest/data_dictionaries/pudl_db.html#coalmine-eia923), [browse the table](https://data.catalyst.coop/pudl/coalmine_eia923)). One of the columns in that table is the MSHA Mine ID. The `coalmine_eia923` table is linked to the `fuel_receipts_costs_eia923` table ([data dictionary](https://catalystcoop-pudl.readthedocs.io/en/latest/data_dictionaries/pudl_db.html#fuel-receipts-costs-eia923), [browse the table](https://data.catalyst.coop/pudl/fuel_receipts_costs_eia923)).
* Additional, more complete and authoratative coal mine data can be found on [the MSHA website](https://arlweb.msha.gov/OpenGovernmentData/OGIMSHA.asp). MSHA dataset 13, aptly named the ["Mines Data Set" (ZIP)](https://arlweb.msha.gov/OpenGovernmentData/DataSets/Mines.zip), and it uses the same MSHA Mine ID that shows up in `coalmine_eia923` 

**The goal of this interview question is to extract the MSHA Mines Data Set and integrate it with the `coalmine_eia923` table.**

## Part A: Extract and Clean the MSHA Mine Data Set

### Extract

* Design and implement a function or class that can be used to extract the [MSHA Mines Data Set (ZIP)](https://arlweb.msha.gov/OpenGovernmentData/DataSets/Mines.zip).
* This function or class should also be adaptable to extracting the other similarly formatted data sources available from the MSHA website. E.g. the [Controller/Operator History (ZIP)](https://arlweb.msha.gov/OpenGovernmentData/DataSets/ControllerOperatorHistory.zip) or [Employment/Production Data Set (ZIP)](https://arlweb.msha.gov/OpenGovernmentData/DataSets/MinesProdYearly.zip).
* The input to this function should be a URL or local path to the published zipfile.
* The output should be a pandas dataframe.

### Transform/Clean
* Take the extracted MSHA Mines data frame and impose some order on it, in preparation for loading it into a well-normalize database and integrating it with the EIA 923 data referenced above.
* Clean the columns you think are required to define a clear structure for the data.
* Identify which columns in the MSHA data correspond to the columns that appear in the `coalmine_eia923` table, and ensure that they are usable for analysis.
* Feel free to clean any other columns you think would be helpful from an advocacy or research perspective, but you don't need to clean every column.
* Any columns you've cleaned should end up with well-defined data types.

### Some questions to keep in mind
* What is the structure of the data? What information do you expect to be unique?
* If you were integrating this dataset into a database, what information could/should be noramlized into separate tables?

### Some hints
* The MSHA dataset has a [Definitions File](https://arlweb.msha.gov/OpenGovernmentData/DataSets/Mines_Definition_File.txt) with column type and description information. 
* You'll need to use Latin character encoding when extracting the .txt files.

## Part B: Prepare the `coalmine_eia923` table for integration with the MSHA Mines Data Set
You're welcome to use 3rd party libraries for this porition. Your solution doesn't need to be perfect but you should understand its limitations and potential improvements.

* Both the EIA 923 and MSHA Mines datasets contain MSHA mine IDs. Unfortunately, the `coalmine_eia923` table contains both duplicate and missing MSHA ids.
* Identify which rows in `coalmine_eia923` should have MSHA Mine IDs, and which should not.
* If there are records which should have an MSHA Mine ID but don't, try and fill them in based on data from either the `coalmine_eia923` or MSHA Mines data.
* Try and identify duplicate records in the `coalmine_eia923` table and deduplicate them.

Ideally at the end of this process, it should be possible to join the MSHA Mines Data with the `coalmines_eia923` data, providing much more information about the mines associated with fuel deliveries reported in the `fuel_receipts_costs_eia923` table.

### Some questions to keep in mind
* Which fields are the most reliable?
* Which fields are most helpful for identifying unique records?
* What kind of records should the `coalmine_eia923` table really contain? Could it have a single record associated with each MSHA Mine ID? Are all of the data columns that show up `coalmine_eia923` also available from the MSHA Mines data?
* Is there any valuable information getting lost when you deduplicate the records in the `coalmine_eia923` table? If so, what kind of information is it?
* Would it be possible to replace the `coalmine_eia923` table entirely with information from the MSHA Mines Data Set?
* If `coalmine_eia923` was replaced, how would you link the MSHA mine data to the individual fuel deliveries reported in the `fuel_receipts_costs_eia923` table?

### Some hints
* Sometimes, the original form respondents have to fill out can be helpful in understanding data quality issues due to data entry. [Here](https://www.eia.gov/Survey/#eia-923) is the EIA 923 Form.

# Set up access to the PUDL database
* You can download a copy of our PUDL SQLite DB from: https://data.catalyst.coop/pudl.db
* Substitute the path to that file on your system below:

In [ ]:
import sqlalchemy as sa
import pandas as pd

PUDL_DB_PATH = "/path/to/your/copy/of/pudl.db"

pudl_engine = sa.create_engine(f"sqlite:///{PUDL_DB_PATH}")

In [ ]:
# Read the coalmine_eia923 table:
coalmine_eia923 = pd.read_sql("coalmine_eia923", pudl_engine)
coalmine_eia923.info()

In [ ]:
# Read the fuel_receipts_costs_eia923 table:
fuel_receipts_costs_eia923 = pd.read_sql("fuel_receipts_costs_eia923", pudl_engine)
fuel_receipts_costs_eia923 .info()